In [286]:
import numpy as np
import os
from six.moves import urllib

# Brute data from different sources:
# https://ai-project-inflation.s3.amazonaws.com/
DOWNLOAD_ROOT = "https://ai-project-inflation.s3.amazonaws.com/"
INFLATION_PATH = os.path.join("datasets", "inflation")

def fetch_housing_data(file_name):
    inflation_url= DOWNLOAD_ROOT+file_name
    inflation_path=INFLATION_PATH
    if not os.path.isdir(inflation_path):
        os.makedirs(inflation_path)
    inflation_path = os.path.join(inflation_path, file_name)
    urllib.request.urlretrieve(inflation_url, inflation_path)

def load_inflation_data(file_name):
    fetch_housing_data(file_name)
    return os.path.join(INFLATION_PATH,file_name)

In [287]:
import pandas as pd
import datetime
def truncate_date(date):
  ts = pd.to_datetime(str(date))
  return ts.strftime('%Y-%m')

def parsing_string_to_date(date):
  format = "%B %Y"
  return truncate_date(datetime.datetime.strptime(date, format))

In [288]:
brent_crude_prices = pd.read_excel(load_inflation_data("Brent_crude_prices.xls"), sheet_name=1, header=2)
brent_crude_prices["Date"] = brent_crude_prices["Date"].apply(truncate_date)
brent_crude_prices.head()
# brent_crude_prices.info()

,Date,Europe Brent Spot Price FOB (Dollars per Barrel)
0,1987-05,18.58
1,1987-06,18.86
2,1987-07,19.86
3,1987-08,18.98
4,1987-09,18.31


In [289]:
!pip install openpyxl
prime_rate = pd.read_excel(load_inflation_data("Prime_Rate_History-WOWA.xlsx"))
prime_rate["Date"] = prime_rate["Date"].apply(parsing_string_to_date)
prime_rate = prime_rate[["Date","Prime Rate" ]]
prime_rate.head()

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


,Date,Prime Rate
0,1935-01,5.50
1,1936-01,5.21
2,1937-01,5.00
3,1938-01,5.00
4,1939-01,5.00


In [290]:
bond_rate = pd.read_csv(load_inflation_data("StlouisfedCAD10yearbondrate.csv"))
bond_rate.rename(columns={'DATE': 'Date'}, inplace=True)
bond_rate["Date"] = bond_rate["Date"].apply(truncate_date)
bond_rate.head()

,Date,IRLTLT01CAM156N
0,2000-01,6.491500
1,2000-02,6.338571
2,2000-03,6.028261
3,2000-04,5.924211
4,2000-05,6.215000


In [291]:
fx_rates = pd.read_csv(load_inflation_data("FX_RATES_MONTHLY_CSV.csv"))
fx_rates["Date"] = fx_rates["Date"].apply(truncate_date)
fx_rates.head()

,Date,1 USD
0,1990-01,1.172038
1,1990-02,1.196479
2,1990-03,1.179986
3,1990-04,1.164095
4,1990-05,1.174700


In [292]:
manufacturing_rate = pd.read_csv(load_inflation_data("Stlouisfeb_hourmanurate.csv"))
manufacturing_rate.rename(columns={'DATE': 'Date'}, inplace=True)
manufacturing_rate["Date"] = manufacturing_rate["Date"].apply(truncate_date)
manufacturing_rate.head()
print(load_inflation_data("Stlouisfeb_hourmanurate.csv"))

datasets/inflation/Stlouisfeb_hourmanurate.csv


In [293]:
money_supply = pd.read_csv(load_inflation_data("Stlousisfed_Canadain_Money_supply.csv"))
money_supply.rename(columns={'DATE': 'Date'}, inplace=True)
money_supply["Date"] = money_supply["Date"].apply(truncate_date)
money_supply.head()

,Date,MANMM101CAM189S
0,2000-01,2.166880e+11
1,2000-02,2.216730e+11
2,2000-03,2.250820e+11
3,2000-04,2.284850e+11
4,2000-05,2.281010e+11


In [294]:
fed_fx_rates = pd.read_csv(load_inflation_data("stlousisfedFXRates.csv"))
fed_fx_rates.rename(columns={'DATE': 'Date'}, inplace=True)
fed_fx_rates["Date"] = fed_fx_rates["Date"].apply(truncate_date)
fed_fx_rates.head()

,Date,CCUSMA02CAM618N
0,2000-01,1.448226
1,2000-02,1.450341
2,2000-03,1.460354
3,2000-04,1.468305
4,2000-05,1.494781


In [295]:
cpi_inflation = pd.read_csv(load_inflation_data("1810025601-CPI_and_Inflation_ydata.csv"), header=8, nrows=9, skiprows={10,14})
cpi_inflation.rename(columns={'Alternative measures': 'Date'}, inplace=True)
# cpi_inflation.index = cpi_inflation.iloc[:, 0]


cpi_inflation_inverted = pd.DataFrame.transpose(cpi_inflation, copy=False)

# pd.DataFrame.transpose(cpi_inflation, copy=True)
# cpi_inflation.head()

# cpi_inflation_inverted = cpi_inflation_inverted.drop(cpi_inflation_inverted.index[[0]])
# cpi_inflation_inverted.reset_index()
# cpi_inflation_inverted["Date"] = cpi_inflation_inverted["Date"].apply(parsing_string_to_date)
cpi_inflation_inverted.head(10)
# print(cpi_inflation_inverted)




,0,1,2,3,4,5,6,7,8
Date,Measure of core inflation based on a factor mo...,Measure of core inflation based on a weighted ...,Measure of core inflation based on a trimmed m...,"Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin..."
May 2000,1.6,1.4,1.6,95.5,95.5,95.4,95.4,95.4,95.3
June 2000,1.7,1.6,1.8,95.8,95.8,96.0,95.6,95.7,95.8
July 2000,1.8,1.6,1.8,95.8,95.9,96.3,95.7,95.7,96.1
August 2000,1.9,1.7,1.9,96.0,96.0,96.2,95.9,95.9,96.1
September 2000,1.8,1.6,1.8,96.2,96.2,96.6,96.0,96.0,96.5
October 2000,1.7,1.7,1.9,96.3,96.3,96.8,96.2,96.2,96.8
November 2000,1.9,1.8,2.0,96.4,96.4,97.1,96.4,96.4,97.2
December 2000,1.9,1.9,2.1,96.4,96.4,97.2,96.7,96.7,97.4
January 2001,2.2,2.0,2.2,96.3,96.3,96.8,96.7,96.7,97.2


In [296]:
tsx_data = pd.read_csv(load_inflation_data("TSX_data_for_20_years.csv"))
tsx_data["Date"] = tsx_data["Date"].apply(truncate_date)
tsx_data = tsx_data[["Date", "Close"]]
tsx_data.head()

,Date,Close
0,2000-01,8481.099609
1,2000-02,9129.000000
2,2000-03,9462.400391
3,2000-04,9347.599609
4,2000-05,9252.000000


In [297]:
dataList = [brent_crude_prices,prime_rate,bond_rate, fx_rates, manufacturing_rate, money_supply, fed_fx_rates, tsx_data]
# cpi_inflation_inverted
data_merged = pd.DataFrame()
initialIteration = True
for single_data in dataList:
  if initialIteration:
    data_merged = single_data.copy()
    initialIteration = False
  else:
    data_merged= data_merged.merge(single_data, left_index=True, right_index=True)
    data_merged.pop('Date_y')
    data_merged.rename(columns={'Date_x': 'Date'}, inplace=True)

In [298]:
data_merged.head()

,Date,Europe Brent Spot Price FOB (Dollars per Barrel),Prime Rate,IRLTLT01CAM156N,1 USD,LCEAMN01CAM189S,MANMM101CAM189S,CCUSMA02CAM618N,Close
0,1987-05,18.58,5.50,6.491500,1.172038,18.25,2.166880e+11,1.448226,8481.099609
1,1987-06,18.86,5.21,6.338571,1.196479,18.33,2.216730e+11,1.450341,9129.000000
2,1987-07,19.86,5.00,6.028261,1.179986,18.23,2.250820e+11,1.460354,9462.400391
3,1987-08,18.98,5.00,5.924211,1.164095,18.27,2.284850e+11,1.468305,9347.599609
4,1987-09,18.31,5.00,6.215000,1.174700,18.42,2.281010e+11,1.494781,9252.000000
